In [ ]:
# EV Routing pakcage
from ev_routing import EVRouting

# Visualization packages
import plotly.graph_objects as go

import os


# Initializing EVRouting
bl = ( 52.50, 13.34 ) # Bottom left corner coordinate
tr = ( 52.53, 13.43 ) # Top right corner coordinate

area = [ bl[0], bl[1], tr[0], tr[1] ]
evr = EVRouting( area )


# Initializing bapbox
mapbox_access_token = os.environ['mapbox_access_token']

latitudes = [ v['lat'] for v in evr.v.values() ]
longitudes = [ v['lon'] for v in evr.v.values() ]

node_ids = [ v['id'] for v in evr.v.values() ]
node_labels = [ str(i) for i in range(len(evr.v)) ]

In [ ]:
nodes_map = go.Figure()

nodes_map.add_trace( go.Scattermapbox(
    lat = latitudes,
    lon = longitudes,
    mode = 'markers',
    marker = go.scattermapbox.Marker(
        size = 6,
        color = 'rgb(242, 177, 172)',
        opacity=0.7
    ),
    text = node_labels,
    hoverinfo = 'text'
))

nodes_map.update_layout(
    autosize = True,
    hovermode = 'closest',
    showlegend = False,
    mapbox = go.layout.Mapbox(
        accesstoken = mapbox_access_token,
        bearing = 0,
        center = go.layout.mapbox.Center(
            lat = evr.map_center[0],
            lon = evr.map_center[1]
        ),
        pitch = 0,
        zoom = 12,
        style = 'light'
    ),
)

nodes_map.show()

In [ ]:
s = node_ids[13245]
t = node_ids[6511]
bs = 1e6

bt, SoC, trace = evr.dijkstra( s, t, bs )
bt

In [ ]:
from math import log10

trace_resolution = 50
SoC_resolution = ( 20, 15 )

trace_map = go.Figure()

steps = int( len(trace) / 100 ) + 1

x_px = ( tr[0] - bl[0] ) / SoC_resolution[0]
y_px = ( tr[1] - bl[1] ) / SoC_resolution[1]

regions = {}


total_weight = 0
for soc in SoC.values():
    if soc['prev'] == -1: continue
    
    total_weight += 1
    
    i = int( ( evr.v[ soc['prev'] ]['lat'] - bl[0] ) / x_px )
    j = int( ( evr.v[ soc['prev'] ]['lon'] - bl[1] ) / y_px )
    key = int( .5 * (i + j) * (i + j + 1) + j )
    
    if key in regions:
        regions[key]['np'] += 1
    else:
        regions[key] = { 'lat': (i + .5) * x_px + bl[0], 'lon': (j + .5) * y_px + bl[1], 'np': 1 }

        
for region in regions.values():
    region['np'] /= float( total_weight )

min_log10 = min( [ log10(region['np']) for region in regions.values() ] )


for region in regions.values():
    w = log10( region['np'] ) / min_log10
    c = str( min( int( w * 255), 255 ) )
    rgb = 'rgb(' + c + ', ' + c + ', ' + c + ')'
    
    trace_map.add_trace( go.Scattermapbox(
        lat = [ region['lat'] ],
        lon = [ region['lon'] ],
        mode = 'markers',
        marker = go.scattermapbox.Marker(
            size = 1 / w * 20,
            color = rgb,
            opacity = min( 1 / w * 0.7, 0.7 )
        )
    ))



pv = evr.v[s]
for vid in trace[ ::steps ]:
    lat = [ pv['lat'], evr.v[vid]['lat'] ]
    lon = [ pv['lon'], evr.v[vid]['lon'] ]
    pv = evr.v[vid]

    
    trace_map.add_trace( go.Scattermapbox(
        lat = lat,
        lon = lon,
        mode = 'lines',
        line = go.scattermapbox.Line(
            width = 6,
            color = 'rgb(87, 160, 211)'
        )
    ))
    
trace_map.add_trace( go.Scattermapbox(
    lat = [ evr.v[s]['lat'], evr.v[t]['lat'] ],
    lon = [ evr.v[s]['lon'], evr.v[t]['lon'] ],
    mode = 'markers',
    marker = go.scattermapbox.Marker(
        size = 10,
        color = 'rgb(242, 177, 172)',
    ),
    text = [ 'Start', 'End' ],
    hoverinfo = 'text'
))



trace_map.update_layout(
    autosize = True,
    hovermode = 'closest',
    showlegend = False,
    mapbox = go.layout.Mapbox(
        accesstoken = mapbox_access_token,
        bearing = 0,
        center = go.layout.mapbox.Center(
            lat = evr.map_center[0],
            lon = evr.map_center[1]
        ),
        pitch = 0,
        zoom = 12,
        style = 'light'
    ),
)

trace_map.show()